In [223]:
import numpy as np
import pandas as pd
import time
from datetime import datetime

In [229]:
order = pd.read_csv("delivery_orders_march.csv")

In [230]:
order.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [231]:
order["1st_deliver_attempt"] = order["1st_deliver_attempt"].astype("int")
order['2nd_deliver_attempt'].fillna(0,inplace=True)
order["2nd_deliver_attempt"] = order["2nd_deliver_attempt"].astype("int")

In [232]:
order["pick"] = order["pick"].apply(lambda x: datetime.fromtimestamp(x))
order["1st_deliver_attempt"] = order["1st_deliver_attempt"].apply(lambda x: datetime.fromtimestamp(x))
order["2nd_deliver_attempt"] = order["2nd_deliver_attempt"].apply(lambda x: datetime.fromtimestamp(x))

In [234]:
def check_city(item):
    if "metro manila" in item.lower():
        return("Metro Manila")
    elif "luzon" in item.lower():
        return("Luzon")
    elif "visayas" in item.lower():
        return("Visayas")
    elif "mindanao" in item.lower():
        return("Mindanao")

In [235]:
order["buyeraddress"] = order["buyeraddress"].apply(check_city)
order["selleraddress"] = order["selleraddress"].apply(check_city)

In [236]:
def check_sla(order):
    if order["buyeraddress"] == "Metro Manila" and order["selleraddress"] == "Metro Manila":
        return(3)
    elif order["buyeraddress"] == "Metro Manila" and order["selleraddress"] == "Luzon":
        return(5)
    elif order["buyeraddress"] == "Luzon" and order["selleraddress"] == "Metro Manila":
        return(5)
    elif order["buyeraddress"] == "Luzon" and order["selleraddress"] == "Luzon":
        return(5)
    else:
        return(7)

In [237]:
order["sla"] = order.apply(check_sla,axis=1)

In [281]:
def check_late(order):
    duration = np.busday_count(order["pick"].date(),order["1st_deliver_attempt"].date(),weekmask="1111110",holidays=["2020-03-08","2020-03-25","2020-03-30","2020-03-31"])
    duration2 = np.busday_count(order["1st_deliver_attempt"].date(),order["2nd_deliver_attempt"].date(),weekmask="1111110",holidays=["2020-03-08","2020-03-25","2020-03-30","2020-03-31"])
    # 1 attempt
    if order["2nd_deliver_attempt"] == "1970-01-01 07:00:00":
        if duration > order["sla"]:
            return(1)
        else:
            return(0)
    # 2 Attempt
    else:
        # 1st attempt late
        if duration > order["sla"]:
            return(1)
        #1st attempt not late
        else:
            #2nd attempt late
            if duration2 > 3:
                return(1)
            #2nd attempt not late
            else:
                return(0)

In [286]:
order.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,sla
0,2215676524,2020-03-02 15:39:57,2020-03-05 12:09:18,1970-01-01 07:00:00,Metro Manila,Metro Manila,3
1,2219624609,2020-03-04 15:19:28,2020-03-06 09:53:56,2020-03-10 07:02:56,Metro Manila,Metro Manila,3
2,2220979489,2020-03-04 14:20:34,2020-03-06 08:56:19,1970-01-01 07:00:00,Metro Manila,Metro Manila,3
3,2221066352,2020-03-05 21:36:56,2020-03-07 11:45:41,1970-01-01 07:00:00,Metro Manila,Metro Manila,3
4,2222478803,2020-03-04 17:38:25,2020-03-06 14:41:40,1970-01-01 07:00:00,Luzon,Metro Manila,5


In [283]:
late = order.apply(check_late,axis=1)

In [287]:
submit = pd.DataFrame({
    "orderid" : order["orderid"],
    "is_late" : late
})

In [291]:
submit["is_late"].value_counts(normalize=True)

0    0.759826
1    0.240174
Name: is_late, dtype: float64

In [292]:
submit.to_csv("submit.csv", index=False)

In [293]:
submit

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0
...,...,...
3176308,31504087640510,0
3176309,31504147352227,0
3176310,31504462290482,0
3176311,31504851495943,0
